In [22]:
import keras
from keras import layers
import numpy as np
import tensorflow as tf

In [112]:
FINAL_NUM = 20
FINAL_SIZE = 32
BATCH_SIZE = 2
IMG_SIZE = 640

# offset constant for centery
COL_IDX = np.array([i for i in range(FINAL_NUM)], dtype=np.float32)
COL_IDX = np.tile(COL_IDX, (BATCH_SIZE, FINAL_NUM, 1)) * FINAL_SIZE

# offset constant for centerx
ROW_IDX = np.transpose(COL_IDX, axes=[0,2,1])

# fill in all other indexes with 0
ADD_CENTER = np.zeros(shape=(BATCH_SIZE, FINAL_NUM, FINAL_NUM, 5), dtype=np.float32)
ADD_CENTER[:,:,:,1] = ROW_IDX
ADD_CENTER[:,:,:,2] = COL_IDX

# constant tensor to multiply with centerx/y coords
MULT_CENTER = np.ones(shape=(BATCH_SIZE, FINAL_NUM, FINAL_NUM, 5), dtype=np.float32)
MULT_CENTER[:,:,:,1:3] = FINAL_SIZE-1

# constant tensor to div centery x/y coords
DIV_CENTER = np.ones(shape=(BATCH_SIZE, FINAL_NUM, FINAL_NUM, 5), dtype=np.float32)
DIV_CENTER[:,:,:,1:3] = IMG_SIZE

ADD_CENTER = tf.convert_to_tensor(ADD_CENTER, dtype=tf.float32)
MULT_CENTER = tf.convert_to_tensor(MULT_CENTER, dtype=tf.float32)
DIV_CENTER = tf.convert_to_tensor(DIV_CENTER, dtype=tf.float32)

In [6]:
inputs = keras.Input((20,20,1))
outputs = layers.Conv2D(5, kernel_size=(1,1), strides=(1,1), activation='sigmoid')(inputs)
model = keras.Model(inputs=inputs, outputs=outputs)

In [104]:
CE = keras.losses.categorical_crossentropy

def custom_loss(y_true, y_pred):

    # categorical cross entropy using first index of all 20 predictions
    y_true_conf = tf.reshape(y_pred[:,:,:,0], [-1, FINAL_NUM*FINAL_NUM])
    y_pred_conf = tf.reshape(y_pred[:,:,:,0], [-1, FINAL_NUM*FINAL_NUM])
    #id_accuracy = CE(y_true, y_pred)

    # converting format from offset centerx/y -> centerx/y
    y_pred = (y_pred * MULT_CENTER + ADD_CENTER) / DIV_CENTER
    test = CE(y_true, y_pred)

    return test

In [105]:
model.compile(
    loss=custom_loss,
    optimizer=keras.optimizers.Adam()
)

In [20]:
x_train = np.random.rand(10, 20, 20)
y_train = np.random.rand(10, 5)

In [92]:
FINAL_NUM=5
BATCH_SIZE=2
FINAL_SIZE = 32

# offset constant for centery
COL_IDX = np.array([i for i in range(FINAL_NUM)], dtype=np.float32)
COL_IDX = np.tile(COL_IDX, (BATCH_SIZE, FINAL_NUM, 1)) * FINAL_SIZE

# offset constant for centerx
ROW_IDX = np.transpose(COL_IDX, axes=[0,2,1])

# fill in all other indexes with 0
ADD_CENTER = np.zeros(shape=(BATCH_SIZE, FINAL_NUM, FINAL_NUM, 5))
ADD_CENTER[:,:,:,1] = ROW_IDX
ADD_CENTER[:,:,:,2] = COL_IDX

# TEST

In [106]:
y_pred = model.predict(x_train)

1/1 [==============================] - 0s 99ms/step


In [ ]:
model.fit(x_train, y_train, batch_size=2, epochs=2)